In [4]:
#the code below illustrate how descriptor funcion as a class field of client class
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        print("\n#########################################")
        print("__get__ is being called")
        print(f"{self.__class__.__name__} \n{instance}\n{owner}")
        print("###################################################\n")
        return instance

class ClientClass:
    descriptor=DescriptorClass()
    
    
client=ClientClass()

client.descriptor


#########################################
__get__ is being called
DescriptorClass 
<class '__main__.ClientClass'>
###################################################



In [7]:
#this example examplains the difference a descriptor being called from class or instancce
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return f"{self.__class__.__name__}.{owner.__name__}"
        return f"value for {instance}"
    
class ClientClass:
    descriptor=DescriptorClass()
    
print(ClientClass.descriptor)
client=ClientClass()
print(client.descriptor)




DescriptorClass.ClientClass
value for <__main__.ClientClass object at 0x0000012B4A8C86A0>


In [3]:

class TextField:
    def __set_name__(self, owner, name):
        print("__set__name was called")
        print(f"{owner=},{name=}")
        self.name = name

    def __set__(self, instance, value):
        print("__set__ was called!!")
        if not isinstance(value, str):
            raise AttributeError("must be str")
        instance.__dict__[self.name] = value

    def __get__(self, instance, owner):
        print("get was callled")
        return instance.__dict__[self.name]


class Book:
    title = TextField()


book = Book()
book.title

__set__name was called
owner=<class '__main__.Book'>,name='title'
get was callled


KeyError: 'title'

In [5]:
book.title="python for ml learning"

__set__ was called!!


In [17]:
from typing import Callable, Any


class Validation:
    def __init__(self, validation_function: Callable[[Any], bool], error_msg: str):
        self.validation_function = validation_function
        self.error_msg = error_msg

    def __call__(self, value):
        """
        raise error when any of validation_functions fail

        Parameters
        ----------
        value : [type]
            [description]

        Raises
        ------
        ValueError
            [description]
        """
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")


class Field:
    def __init__(self, *validations):
        self._name = None
        self.validations = validations

    def __set_name__(self, owner, name):
        print("set_name was called!!")
        print(f"{owner=},{name=}")
        self._name = name

    def __get__(self, instance, owner):
        print("\n__get__ is called\n")
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def validate(self, value):
        for validation in self.validations:
            validation(value)

    def __set__(self, instance, value):
        print("__set__ was called")
        print(f"{instance=} ,{value=}")
        self.validate(value)
        instance.__dict__[self._name] = value


class ClientClass:
    descriptor = Field(Validation(lambda x: isinstance(x, (int, float)), "is not a number"),
                       Validation(lambda x: x >= 0, "is not >=0"))

a=ClientClass()


set_name was called!!
owner=<class '__main__.ClientClass'>,name='descriptor'


In [19]:
a.descriptor=123
print(type(a.descriptor))

__set__ was called
instance=<__main__.ClientClass object at 0x00000171536E0F70> ,value=123

__get__ is called

<class 'int'>


In [23]:
from enum import Enum

Permission = Enum("Permission", "admin email helpdesk")


class ProtectedAttribute:
    def __init__(self, requires_role: Permission = None):
        self.permission_required = requires_role
        self._name = None

    def __set__(self, instance, value):
        if value is None:
            raise ValueError(f"{self._name} cant be set to None")
        instance.__dict__[self._name] = value

    def __set_name__(self, owner, name):
        print("__set_name is being called")
        self._name = name

    def __delete__(self, instance):
        if self.permission_required in instance.permissions:
            instance.__dict__[self._name] = None
        else:
            raise ValueError(
                f"User {user!s} doesnt have {self.permission_required}")


class User:
    email = ProtectedAttribute(requires_role=Permission.admin)

    def __init__(self, username: str, email: str, permission_list: list = None):
        self.username = username
        self.email = email
        self.permissions = permission_list or []

    def __str__(self):
        return self.username


__set_name is being called


In [25]:
from enum import Enum

Permission = Enum("Permission", "admin user")


class ProtectedAttribute:
    def __init__(self, requires_role: Permission = None):
        self.permission_required = requires_role
        self._name = None

    def __set__(self, instance, value):
        if value is None:
            raise ValueError(f"{self._name} cant be set to None")
        instance.__dict__[self._name] = value

    def __set_name__(self, owner, name):
        print("__set_name is being called")
        self._name = name

    def __delete__(self, instance):
        print("__delete__ is being called!")
        if self.permission_required in instance.permissions:
            instance.__dict__[self._name] = None
        else:
            raise ValueError(
                f"User {instance!s} doesnt have {self.permission_required}")


class User:
    email = ProtectedAttribute(requires_role=Permission.admin)

    def __init__(self, username: str, email: str, permission_list: list = None):
        self.username = username
        self.email = email
        self.permissions = permission_list or []

    def __str__(self):
        return self.username

    
admin=User("root","root@gmail.com",[Permission.admin])
del admin.email


__set_name is being called
__delete__ is being called!


In [32]:
class NonDataDescriptor:
    def __get__(self,instance,owner):
        print("get is being called")
        print(f"{instance=}\n{owner=}")
        if instance is None:
            return self
        return 42

class ClientClass:
    descriptor=NonDataDescriptor()
    
client=ClientClass()

print(client.descriptor)
print(f"{vars(client)=}")
client.descriptor=123
print(f"{vars(client)=}")
print(client.descriptor)

get is being called
instance=<__main__.ClientClass object at 0x00000171537C00A0>
owner=<class '__main__.ClientClass'>
42
vars(client)={}
vars(client)={'descriptor': 123}
123


In [38]:
#save the history of the current city attribute in the list
#with descriptor ,codes below looks complicated,and actully it is.
#So it is suted for large API or libraries,not for simple scripts for boaring tasks!!
from typing import ValuesView


class HistoryTracedAttribute:
    def __init__(self, trace_attribute_name: str):
        self.trace_attribute_name = trace_attribute_name
        self._name = None

    def __set_name__(self, owner, name):
        print("_set_name is being called")
        self._name = name

    def __set__(self, instance, value):
        print("__set__ is being called!!")
        self._track_change_in_value_for(instance, value)
        instance.__dict__[self._name] = value

    def __get__(self, instance, owner):
        print("__get is being called")
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def _track_change_in_value_for(self, instance, value):
        self._set_default(instance)
        if self._needs_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value)

    def _needs_to_track_change(self, instance, value) -> bool:
        try:
            current_value = instance.__dict__[self._name]
        except KeyError:
            return True
        return value != current_value

    def _set_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, [])


class Traveler:
    current_city = HistoryTracedAttribute("cities_visited")

    def __init__(self, name: str, current_city: str):
        self.name = name
        self.current_city = current_city

        
        
        
traveler=Traveler("Hoge","tokyo")
print(traveler.current_city)
print(vars(traveler))
traveler.current_city="osaka"

traveler.__dict__["cities_visited"]

_set_name is being called
__set__ is being called!!
__get is being called
tokyo
{'name': 'Hoge', 'cities_visited': ['tokyo'], 'current_city': 'tokyo'}
__set__ is being called!!


['tokyo', 'osaka']